# Online-Food Ordering System

# Relational Database Management Systems

## Performing CRUD(Create,Read,Update,Delete) Operations of persistent storage using python with ORM(SQLAlchemy)

## I.Creating the database Using SQLAlchemy ORM

In [1]:
# Import sqlalchemy
import sqlalchemy



# Check version
sqlalchemy.__version__ 

'1.3.19'

In [2]:
# Imports the declarative_base object, which connects the database engine to the SQLAlchemy functionality of the models.
from sqlalchemy.ext.declarative import declarative_base

# Imports the Column, Integer, String, ForeignKey, and Table classes from SQLAlchemy, which are used to help define the model attributes.
from sqlalchemy import Column, ForeignKey, Float , Integer, String, Numeric, DateTime, ForeignKey, CHAR, Table, LargeBinary, DECIMAL

# Imports the relationship() object, which are used to create the relationships between objects.
from sqlalchemy.orm import relationship

# Imports create_engine
from sqlalchemy import create_engine



In [3]:
# Creates the Base class, which is what all models inherit from and how they get SQLAlchemy ORM functionality.
Base=declarative_base()

# create_engine() function is called to set up an engine object which is subsequently used to perform SQL operations. 
# Create an engine that stores data in the local directory. "///" is the relative path (The SQLite database is created in the same folder as the Jupyter script)
# The first parameter is the name of the database.
# The second parameter (echo) is set to true; this will generate the activity log
engine = create_engine('sqlite:///BEMM459_ASSIGNMENT.db', echo=True)

"""In ORM, the configurational process starts by describing the database tables and then by defining classes which will be mapped to those tables. 
   In SQLAlchemy, these two tasks are performed together. 
   A class in Declarative must have a __tablename__ attribute, and at least one Column which is part of a primary key. 
   All the columns in all the tables are set to nullable=False implying all the records are mandatory
"""

"""During construction of this database three types of relationships are being used, namely:
   1)One to One(1-1) relationship: 
     One To One is essentially a bidirectional relationship with a scalar attribute on both sides.
     Within the ORM, “one-to-one” is considered as a convention where the ORM expects that only one related row will exist for any parent row.
   2)One to Many(1-*) relationship: 
     A one to many relationship places a foreign key on the child table referencing the parent. 
     relationship() is then specified on the parent, as referencing a collection of items represented by the child.
   3)Many to Many(*-*) relationship:
     Table uses the MetaData object associated with the declarative base class, so that the ForeignKey directives can locate the remote tables with which to link.
     In our case, we're Defining a Link class as User and Site_info has *-* relationship. It is linked to link table and contains site_info_id1 and user_id1 attributes respectively referencing to primary keys of User and Site_info table.
"""

"""Prevention Strategy used:
        We've used CASCADE for both update and delete on all the foreign keys in all the tables.
        Cascading automatically updates referencing child records.
        Hence, if a record is updated or deleted on a parent class, we're referencing to delete/update that record on all the child classes.
        
# The cascade attribute in relationship function is a comma-separated list of cascade rules which determines how Session operations should be “cascaded” from parent to child. 
# By default: it is False, which means that it is "save-update, merge".
# Change to: "all, delete-orphan" to indicate that related objects should follow along with the parent object in all cases, and be deleted when de-associated.
"""


class Customer(Base):
    """Defines the Customer class model to the Customers database table.
       Customer table contains the below basic information about the customers.
       customer_id : Primary key of the table which autoincrements. Takes only Integer type inputs.
       customer's first,last & middle name :Column containing String types input of the customer's name.
       customer_email : Email address of customer with data-type customer.
       customer_phone_number and customer_landline : Contact numbers of the customers with String input-types.
       profile_image : Profile picture of the customer saved in LargeBinary. 
       This can be a null value as it's not mandatory to have a profile picture.
       Customer_username &customer_password : Columns containing customer login credentials. Data-type is String
       account_status : Account status of the customers in integer type. 0 refers to inactive, 1 refers to active
    """
    
    __tablename__ = 'Customer'
    customer_id = Column(Integer, primary_key=True, autoincrement=True)
    customer_first_name = Column(String(30),nullable=False)
    customer_last_name = Column(String(30),nullable=False)
    customer_middle_name = Column(String(30))
    customer_email = Column(String(50),nullable=False)
    customer_phone_number = Column(String(15),nullable=False)
    customer_landline = Column(String(15))
    profile_image = Column(LargeBinary)
    customer_username = Column(String(30),nullable=False)
    customer_password = Column(String(30),nullable=False)
    account_status = Column(Integer,nullable=False)
    
    ####1-1 relationship with Rating table
    #"backref='customer"" is only a common shortcut for placing a second relationship() onto the Rating mapping, including the establishment of an event listener on both sides(Rating and Customer) which will mirror attribute operations in both directions. 
    #To convert this to “one-to-one”, the “one-to-many” or “collection” side is converted into a scalar relationship using the "uselist=False" flag.
    ratingR = relationship('Rating', backref='customer',uselist=False)
    
    ##1-* relationship with Order table
    #backref='Order' is only a common shortcut for placing a second relationship(customer) onto the Order mapping, including the establishment of an event listener on both sides(Order and Customer) which will mirror attribute operations in both directions. 
    orderR1 = relationship('Order', backref='customer')


"""Many to Many relationship between two tables(User and SIte_info) is achieved by adding an association table such that it has two foreign keys - one from each table’s primary key. 
   User(restaurant Owner) can own many Site_info(Restaurant), and a Site_info(restaurant) can have multiple Users(owners).
"""
class User(Base):
    """Defines the User class model to the User database table.
       user_id : Primary key of the table which is set to autoincrement. Has integer type values.
       full_name : Column with Full name of the owner which is stored as String(VarChar).
       contact_number : Column with Contact number of the restaurant owner stored as String.
       Email address : Column with email contact of the owner stored as String.
       username and password : Admin login credentials for the owner so that they can keep menu details updated regularly.
    """
    __tablename__ = 'User'
    user_id = Column(Integer(), primary_key=True,autoincrement=True,nullable=False)
    full_name = Column(String(100),nullable=False)
    contact_number = Column(String(15),nullable=False)
    email_address = Column(String(50),nullable=False)
    username = Column(String(30),nullable=False)
    password = Column(String(30),nullable=False)
    
    ##Link class relationship with Site_info table
    site_infoR = relationship('Site_info',secondary='link')
    
    ##1-* relationship with Order table
    #backref='user' is only a common shortcut for placing a second relationship(user) onto the Rating mapping, including the establishment of an event listener on both sides(Order and User) which will mirror attribute operations in both directions.
    orderR = relationship('Order', backref = 'user')
    
    ##1-* relationship with Payment_info table
    #"backref='Payment_info' is only a common shortcut for placing a second relationship(user) onto the Rating mapping, including the establishment of an event listener on both sides(Payment_info and User) which will mirror attribute operations in both directions.
    payment_infoR1 = relationship('Payment_info', backref = 'user')


class Site_info(Base):
    """Defines the Site_info class model to the Site_info database table.
       site_info_id : Primary key of the table which is being autoincremented.
       site_name : Primary key of the table which is set to autoincrement
       description:describes the restaurant
       contact_info: contact information restaurant
       address: address of the restaurant
       last_update: last update time and date
       user_id: foreign key of the table referencing User table
    """
    __tablename__ = 'Site_info'
    site_info_id = Column(Integer(), primary_key=True, autoincrement=True, nullable=False)
    site_name = Column(String(30),nullable=False)
    description = Column(String(100))
    contact_info = Column(String(50),nullable=False)
    address = Column(String(100),nullable=False)
    last_update = Column(DateTime)
    user_id = Column(Integer(), ForeignKey("User.user_id", onupdate="CASCADE", ondelete="CASCADE"), nullable=False)
    
    ##Link class relationship with User Table
    userR = relationship('User', secondary = 'link')

class Link(Base):
    """Defining a Link class. 
       As User and Site_info has *-* relationship. It is linked to link table and contains site_info_id1 and user_id1 attributes respectively referencing to primary keys of User and Site_info table.
       site_info_id1: primary key of sites_info table referenced as foreign key
       user_id1: primary key of User table referenced as foreign key
    """
    __tablename__ = 'link'
    site_info_id1 = Column(Integer,ForeignKey('Site_info.site_info_id'), primary_key=True)
    user_id1 = Column(Integer,ForeignKey('User.user_id'), primary_key=True)
    
    
    
class Order(Base):
    """Defines the Order class model to the Order database table.
       customer_id: Primary key of the table which is being autoincremented.
       order_date: date of order placed
       total_amount: total cost of the order 
       order_status : 0-pending, 1-confirmed, 2-cancelled.
       discount_coupon: discount vouchers of the user
       discount_percentage: discount percentage
       amount_after_discount: total amount after the discount
       processed_by: contains restaurant owner who received the payment for the order
       
    """
    __tablename__ = 'Order'
    order_id = Column(Integer(), primary_key=True, nullable=False, autoincrement=True)
    customer_id = Column(Integer(),ForeignKey("Customer.customer_id", onupdate="CASCADE", ondelete="CASCADE"))
    order_date = Column(Integer)
    total_amount = Column(DECIMAL(10,2),nullable=False)
    order_status = Column(Integer(),nullable=False)
    discount_coupon = Column(String)
    discount_percentage = Column(Integer)
    amount_after_discount = Column(DECIMAL(10,2))
    
    processed_by = Column(Integer(),ForeignKey("User.user_id", onupdate="CASCADE", ondelete="CASCADE"),nullable=False)
    
    ####1-1 relationship with payment_info table
    #"backref='order'"" is only a common shortcut for placing a second relationship(order) onto the Payment_info mapping, including the establishment of an event listener on both sides(Payment_info and Order) which will mirror attribute operations in both directions. 
    #To convert this to “one-to-one”, the “one-to-many” or “collection” side is converted into a scalar relationship using the "uselist=False" flag.
    payment_infoR = relationship('Payment_info',backref='order',uselist=False)
    
    ##1-* relationship with order_details
    #backref='order' is only a common shortcut for placing a second relationship(order) onto the Rating mapping, including the establishment of an event listener on both sides(Order_details and Order) which will mirror attribute operations in both directions.
    order_detailsR = relationship('Order_details', backref='order')


class Payment_info(Base):
    """Defines the Payment_info class model to the Payment_info database table.
       Payment_info table contains information on payments done by the customer.
       payment_id : Primary key of the table which is autoincremented and has Interger-type values.
       order_id : Foreign key of the table which references to order_id of the Order table. Has Integer-type values.
       procesed_by : Foreign key of the table which references to user_id of the User table. Has Integer-type values.
       amount : Refers to the total amount paid by the customer. Has decimal(10,2) values-> 2 decimal digits limit after the dot.
       paid_by : Refers to the name of the person who made the payment.
       payment_date : Column having date of the transacion made. Stored in DateTime format.
    """
    
    __tablename__ = 'Payment_info'
    payment_id = Column(Integer(), primary_key=True,autoincrement=True,nullable=False)
    order_id = Column(Integer(),ForeignKey("Order.order_id", onupdate="CASCADE", ondelete="CASCADE"),nullable=False)
    amount = Column(DECIMAL(10,2),nullable=False)
    paid_by = Column(String(50),nullable=False)
    payment_date = Column(DateTime)
    processed_by = Column(Integer(),ForeignKey("User.user_id", onupdate="CASCADE", ondelete="CASCADE"),nullable=False)

# Defines the Menu_type class model to the Menu_type database table.
class Menu_type(Base):
    """Defines the Menu_type class model to the Menu_type database table.
       menu_type_id : Integer-type Column which is a primary key for the table.
       type_name : Column refers to the item-type/menu-type. Inputs String-type.
       description : has descriptions on the menu-type. Inputs String-type.
       
       RELATIONSHIPS : 1-1 with the Menu table
       
       backref='menu_type' is only a common shortcut for placing a second relationship() onto the Menu mapping, including the establishment of an event listener on both sides which will mirror attribute operations in both directions. 
       To convert this to “one-to-one”, the “one-to-many” or “collection” side is converted into a scalar relationship using the uselist=False flag.
    """
    __tablename__ = 'Menu_type'
    menu_type_id = Column(Integer(), primary_key=True, autoincrement=True, nullable=False)
    type_name = Column(String(50),nullable=False)
    description = Column(String(100),nullable=False)
    
    ####1-1 relationship with menu table
    #"backref='menu_type'"" is only a common shortcut for placing a second relationship() onto the Menu mapping, including the establishment of an event listener on both sides(Menu and Menu_type) which will mirror attribute operations in both directions. 
    #To convert this to “one-to-one”, the “one-to-many” or “collection” side is converted into a scalar relationship using the "uselist=False" flag.
    menuR = relationship('Menu', backref='menu_type',uselist=False)


class Menu(Base):
    """Defines the Menu class model to the Menu database table.
       menu_id:Integer-type Column which is a primary key for the table
       menu_name: Name of the menu in strings
       ingredients: Ingredients of the menu
       menu_status:1- available 0- out of stock
       price: prive of each menu item
       menu_type_id: type of the menu where it belongs to
        
    """
    __tablename__ = 'Menu'
    menu_id = Column(Integer(), primary_key=True, nullable=False, autoincrement=True)
    menu_name = Column(String(100),nullable=False)
    menu_image = Column(LargeBinary)
    ingredients = Column(String(500))
    menu_status = Column(Integer(),nullable=False)
    price = Column(DECIMAL(10,2),nullable=False)
    menu_type_id = Column(Integer,ForeignKey("Menu_type.menu_type_id", onupdate="CASCADE", ondelete="CASCADE"),nullable=False)
    
    ##1-* relationship with rating table
    #backref='menu' is only a common shortcut for placing a second relationship() onto the Rating mapping, including the establishment of an event listener on both sides(Rating and Menu) which will mirror attribute operations in both directions.
    ratingR = relationship('Rating', backref='menu')


class Order_details(Base):
    """"Defines the Order_details class model to the Order_details database table.
       order_details_id:Integer-type Column which is a primary key for the table
       order_id: foreign key of the table
       menu_id:foreign key with referential integrity
       amount: amount per serving
       no_of_serving: total number of sevrings of each menu
       total_amount:total amount
    """
    __tablename__ = 'Order_detail'
    order_details_id = Column(Integer(), primary_key=True, nullable=False, autoincrement=True)
    order_id = Column(Integer(),ForeignKey("Order.order_id", onupdate="CASCADE", ondelete="CASCADE"),nullable=False)
    menu_id = Column(Integer(),ForeignKey("Menu.menu_id", onupdate="CASCADE", ondelete="CASCADE"),nullable=False)
    amount = Column(DECIMAL(10,2),nullable=False)
    no_of_serving = Column(Integer(),nullable=False)
    total_amount = Column(DECIMAL(10,2),nullable=False)
    
    ##1-* relationship with Menu table
    #backref='order_details' is only a common shortcut for placing a second relationship(order_details) onto the Rating mapping, including the establishment of an event listener on both sides(Menu and Order_details) which will mirror attribute operations in both directions.
    menuR = relationship('Menu', backref='order_details')


class Rating(Base):
    """Defines the Rating class model to the Rating database table.
       rating_id:Integer-type Column which is a primary key for the table.
       menu_id:foreign key with referential integrity
       score: on a scale of 0-5. Integer type only
       remarks: remarks given by the customer
       date_recorded: date recorded
       customer_id: foreign key referencing to customer id of customer table
    """
    __tablename__ = 'Rating'
    rating_id = Column(Integer(), primary_key=True, nullable=False, autoincrement=True)
    menu_id = Column(Integer(), ForeignKey("Menu.menu_id", onupdate="CASCADE", ondelete="CASCADE"),nullable=False)
    score = Column(Integer(),nullable=False)
    remarks = Column(String(100),nullable=False)
    date_recorded = Column(DateTime)
    customer_id = Column(Integer(),ForeignKey("Customer.customer_id", onupdate="CASCADE", ondelete="CASCADE"),nullable=False)
    



# Each Table object is a member of larger collection known as MetaData and this object is available using the .metadata attribute of declarative base class. 
# The MetaData.create_all() method is, passing in our Engine as a source of database connectivity. For all tables that haven’t been created yet, it issues CREATE TABLE statements to the database.
# Create all tables in the engine. This is equivalent to "Create Table"
Base.metadata.create_all(engine)




2022-04-01 05:33:42,342 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-04-01 05:33:42,345 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 05:33:42,347 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-04-01 05:33:42,348 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 05:33:42,349 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Customer")
2022-04-01 05:33:42,350 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 05:33:42,353 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Customer")
2022-04-01 05:33:42,354 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 05:33:42,355 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("User")
2022-04-01 05:33:42,356 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 05:33:42,357 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("User")
2022-04-01 05:33:42,359 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 05:33:42,36

2022-04-01 05:33:42,593 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 05:33:42,612 INFO sqlalchemy.engine.base.Engine COMMIT
2022-04-01 05:33:42,612 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Rating" (
	rating_id INTEGER NOT NULL, 
	menu_id INTEGER NOT NULL, 
	score INTEGER NOT NULL, 
	remarks VARCHAR(100) NOT NULL, 
	date_recorded DATETIME, 
	customer_id INTEGER NOT NULL, 
	PRIMARY KEY (rating_id), 
	FOREIGN KEY(menu_id) REFERENCES "Menu" (menu_id) ON DELETE CASCADE ON UPDATE CASCADE, 
	FOREIGN KEY(customer_id) REFERENCES "Customer" (customer_id) ON DELETE CASCADE ON UPDATE CASCADE
)


2022-04-01 05:33:42,616 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 05:33:42,636 INFO sqlalchemy.engine.base.Engine COMMIT


## Using pandas to Insert records into the table( From .csv files)

In [4]:
#Adding dummy values into all the tables from .csv files using pandas library
import pandas as pd

#Customer Table( has customer information of the registered customers)
customer ='S:\BEMM459_Group_Assignment\customer.csv'
df=pd.read_csv(customer)

"""Creating a connection to the engine with con=engine command
   if the table exists in the database appending the dataframe 'df' to the table
   Making sure external index is not added to the table as the database is already created.
"""
df.to_sql(con=engine,name=Customer.__tablename__, if_exists='append', index=False)




#payment_info(has information on which restaurant owner processed the payment and who paid for the order. Also has order paid by cash or card)
payment_info ='S:\BEMM459_Group_Assignment\payment_info.csv'
df2=pd.read_csv(payment_info)

"""Creating a connection to the engine with con=engine command
   if the table exists in the database appending the dataframe 'df' to the table
   Making sure external index is not added to the table as the database is already created.
"""
df2.to_sql(con=engine,name=Payment_info.__tablename__, if_exists='append', index=False)



#order_details(has details on number of servings placed by the customer on each items)
order_details ='S:\BEMM459_Group_Assignment\order_details.csv'
df4=pd.read_csv(order_details)

"""Creating a connection to the engine with con=engine command
   if the table exists in the database appending the dataframe 'df' to the table
   Making sure external index is not added to the table as the database is already created.
"""
df4.to_sql(con=engine,name=Order_details.__tablename__, if_exists='append', index=False)



#rating(Has information on ratings given by customers on each order placed)
rating ='S:\BEMM459_Group_Assignment\rating.csv'
df5=pd.read_csv(rating)

"""Creating a connection to the engine with con=engine command
   if the table exists in the database appending the dataframe 'df' to the table
   Making sure external index is not added to the table as the database is already created.
"""
df5.to_sql(con=engine,name=Rating.__tablename__, if_exists='append', index=False)



#menu(has all information on all the menu items available from the restaurants and if they're available or not)
menu ='S:\BEMM459_Group_Assignment\menu.csv'
df6=pd.read_csv(menu)

"""Creating a connection to the engine with con=engine command
   if the table exists in the database appending the dataframe 'df' to the table
   Making sure external index is not added to the table as the database is already created.
"""
df6.to_sql(con=engine,name=Menu.__tablename__, if_exists='append', index=False)


#menu_type(has information on the type of food such as dessert,main course, and descriptions of them)
menu_type ='S:\BEMM459_Group_Assignment\menu_type.csv'
df7=pd.read_csv(menu_type)

"""Creating a connection to the engine with con=engine command
   if the table exists in the database appending the dataframe 'df' to the table
   Making sure external index is not added to the table as the database is already created.
"""
df7.to_sql(con=engine,name=Menu_type.__tablename__, if_exists='append', index=False)


#order table(has placed order information such as discount coupon used, total amount and the amount after discount)
order ='S:\BEMM459_Group_Assignment\order.csv'
df3=pd.read_csv(order)
#Calculating the final amount after applying the discount and saving it in a new column
df3['amount_after_discount']=((100-df3['discount_percentage'])*df3['total_amount'])/100

"""Creating a connection to the engine with con=engine command
   if the table exists in the database appending the dataframe 'df' to the table
   Making sure external index is not added to the table as the database is already created.
"""
df3.to_sql(con=engine,name=Order.__tablename__, if_exists='append', index=False)



#Site_info table( Has information on restaurants such as address, contact info)
site_info ='S:\BEMM459_Group_Assignment\site_info.csv'
df1=pd.read_csv(site_info)

"""Creating a connection to the engine with con=engine command
   if the table exists in the database appending the dataframe 'df' to the table
   Making sure external index is not added to the table as the database is already created.
"""
df1.to_sql(con=engine,name=Site_info.__tablename__, if_exists='append', index=False)



2022-04-01 05:33:51,727 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Customer")
2022-04-01 05:33:51,728 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 05:33:51,732 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-04-01 05:33:51,734 INFO sqlalchemy.engine.base.Engine INSERT INTO "Customer" (customer_id, customer_first_name, customer_last_name, customer_middle_name, customer_email, customer_phone_number, customer_landline, profile_image, customer_username, customer_password, account_status) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-04-01 05:33:51,735 INFO sqlalchemy.engine.base.Engine ((1, 'Adria ', 'Snow', 'Lamberg', 'vulputate.ullamcorper@aol.org', '1-398-141-3015', '(573) 377-4240', None, 'JPC63CSG1NX', 'TPC17IEM0RX', 1), (2, 'Dorothy ', 'Vance', 'Underwood', 'morbi@google.net', '(568) 321-4313', '1-241-637-7576', None, 'MPS39WQU8HX', 'EWL29JMF3RV', 1), (3, 'Uta ', 'Benedict', 'Espinoza', 'molestie.sed@icloud.couk', '1-353-143-8156', '1-960-534-2130', None,

FileNotFoundError: [Errno 2] File b'S:\\BEMM459_Group_Assignment\rating.csv' does not exist: b'S:\\BEMM459_Group_Assignment\rating.csv'

## Creating Session and adding records to the SQLite database using SQLAlchemy ORM

In [5]:
# Adding dummy values to User table using SQLAlchemy ORM.

# The Session object enables us to interest with the database (it is a handle to the database)
from sqlalchemy.orm import sessionmaker

# Session class is defined using sessionmaker(), which is bound to the engine object created earlier
Session = sessionmaker(bind = engine)

# session object (objSession) is set-up using the default constructor
objSession = Session()


# using add_all() method of the Session class used to add multiple record
# open database in SQLIte and check if records are added (use ".mode column" and ".header on" options for formatting)
objSession.add_all([
   User( full_name = 'Alan Smith', contact_number='111-111-1111', email_address = 'a1@yahoo.com', username ='a1', password ='aaa1111'),
   User( full_name = 'Barbara Mckinsey', contact_number='222-222-2232' ,email_address = 'h1@yahoo.com', username ='h1', password ='hhh1111'),
   User( full_name = 'Camila Ponting', contact_number='333-333-3333' , email_address = 'p1@yahoo.com', username ='p1', password ='ppp1111'),
   User( full_name = 'David Beckham', contact_number='444-444-4444' , email_address = 's1@yahoo.com', username ='s1', password ='sss1111'),
   User( full_name = 'Elly Styles', contact_number='555-555-5555' , email_address = 's2@yahoo.com', username ='s2', password ='sss2222')
]
)
# Flushes all items and any transaction in progress
# The student object is added to the table 'students'
objSession.commit()

2022-04-01 05:33:56,614 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-04-01 05:33:56,616 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (full_name, contact_number, email_address, username, password) VALUES (?, ?, ?, ?, ?)
2022-04-01 05:33:56,617 INFO sqlalchemy.engine.base.Engine ('Alan Smith', '111-111-1111', 'a1@yahoo.com', 'a1', 'aaa1111')
2022-04-01 05:33:56,622 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (full_name, contact_number, email_address, username, password) VALUES (?, ?, ?, ?, ?)
2022-04-01 05:33:56,624 INFO sqlalchemy.engine.base.Engine ('Barbara Mckinsey', '222-222-2232', 'h1@yahoo.com', 'h1', 'hhh1111')
2022-04-01 05:33:56,625 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (full_name, contact_number, email_address, username, password) VALUES (?, ?, ?, ?, ?)
2022-04-01 05:33:56,626 INFO sqlalchemy.engine.base.Engine ('Camila Ponting', '333-333-3333', 'p1@yahoo.com', 'p1', 'ppp1111')
2022-04-01 05:33:56,627 INFO sqlalchemy.engine.base.E

## II.Displaying records present in the Customer Table (READ Operation)

In [6]:
# Textual SQL
# Literal strings can be used flexibly with Query object by specifying their use with the text()

from sqlalchemy import text
#Displays all the customers with customer_id<=2
for customer in objSession.query(Customer).filter(text("customer_id<=2")):
   print(customer.customer_email)

2022-04-01 05:34:02,159 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-04-01 05:34:02,161 INFO sqlalchemy.engine.base.Engine SELECT "Customer".customer_id AS "Customer_customer_id", "Customer".customer_first_name AS "Customer_customer_first_name", "Customer".customer_last_name AS "Customer_customer_last_name", "Customer".customer_middle_name AS "Customer_customer_middle_name", "Customer".customer_email AS "Customer_customer_email", "Customer".customer_phone_number AS "Customer_customer_phone_number", "Customer".customer_landline AS "Customer_customer_landline", "Customer".profile_image AS "Customer_profile_image", "Customer".customer_username AS "Customer_customer_username", "Customer".customer_password AS "Customer_customer_password", "Customer".account_status AS "Customer_account_status" 
FROM "Customer" 
WHERE customer_id<=2
2022-04-01 05:34:02,162 INFO sqlalchemy.engine.base.Engine ()
vulputate.ullamcorper@aol.org
morbi@google.net


## Taking input from customers to display their required details

In [7]:
# This code block shows the use of one() method, error handling, requesting input from user and binding parameters with string-based SQL

# Required for errorhanding
from sqlalchemy.orm.exc import NoResultFound, MultipleResultsFound
from sqlalchemy import text

# Requesting customer input
customerNum=input("Enter customer id:\n ")


try:
    # The one() method of the Query object fetches all rows, and if there is not exactly one it raises an error MultipleResultsFound or NoResultFound
    # To specify bind parameters with string-based SQL, we use a colon; to specify the values, we use the params() method
    customer = objSession.query(Customer).filter(text("customer_id = :value")).params(value = customerNum).one()
    
    #customer = objSession.query(Customer).filter(text("customer_address LIKE :value")).params(value = customerAdd).one()
    
    #printing just the customers phone number
    print('\nYour phone number as per our record is : \n')
    print(customer.customer_phone_number)
    
except NoResultFound:
    print("Results not found")   
    
except MultipleResultsFound:
    print("Multiple results found")   


Enter customer id:
 1
2022-04-01 05:34:10,438 INFO sqlalchemy.engine.base.Engine SELECT "Customer".customer_id AS "Customer_customer_id", "Customer".customer_first_name AS "Customer_customer_first_name", "Customer".customer_last_name AS "Customer_customer_last_name", "Customer".customer_middle_name AS "Customer_customer_middle_name", "Customer".customer_email AS "Customer_customer_email", "Customer".customer_phone_number AS "Customer_customer_phone_number", "Customer".customer_landline AS "Customer_customer_landline", "Customer".profile_image AS "Customer_profile_image", "Customer".customer_username AS "Customer_customer_username", "Customer".customer_password AS "Customer_customer_password", "Customer".account_status AS "Customer_account_status" 
FROM "Customer" 
WHERE customer_id = ?
2022-04-01 05:34:10,439 INFO sqlalchemy.engine.base.Engine ('1',)

Your phone number as per our record is : 

1-398-141-3015


## Displaying records present in the User Table (READ Operation)

In [8]:
# query object has all() method which returns a resultset in the form of list of objects
resultSet = objSession.query(User).all()

# Displaying all records
for record in resultSet:
    print("ID:",record.user_id, "Name:",record.full_name, "Email Address:",record.email_address, "Username:",record.username)

2022-04-01 05:34:14,553 INFO sqlalchemy.engine.base.Engine SELECT "User".user_id AS "User_user_id", "User".full_name AS "User_full_name", "User".contact_number AS "User_contact_number", "User".email_address AS "User_email_address", "User".username AS "User_username", "User".password AS "User_password" 
FROM "User"
2022-04-01 05:34:14,554 INFO sqlalchemy.engine.base.Engine ()
ID: 1 Name: Alan Smith Email Address: a1@yahoo.com Username: a1
ID: 2 Name: Barbara Mckinsey Email Address: h1@yahoo.com Username: h1
ID: 3 Name: Camila Ponting Email Address: p1@yahoo.com Username: p1
ID: 4 Name: David Beckham Email Address: s1@yahoo.com Username: s1
ID: 5 Name: Elly Styles Email Address: s2@yahoo.com Username: s2


## Doing default join(inner) on two tables(User and Site_info) and displaying the records (READ Operation)

In [9]:
# The Session object enables us to interest with the database (it is a handle to the database)
from sqlalchemy.orm import sessionmaker
sessionObj = Session()

In [10]:
## To construct a simple implicit join between User and Site_info, we can use Query.filter() to equate their related columns together
"""
print("\nMethod 1..\n")
result = sessionObj.query(User).join(Site_info).filter(User.user_id == Site_info.user_id)
for row in result:
    for inv in row.UserR:
        print (row.user_id, row.full_name, inv.address, inv.site_name)
"""
print("\nMethod 2..\n")
#For loop to display the contents while doing an implicit join.
#The above method 1 can also be used which first joins the two table and then prints the required fields.
for c, i in sessionObj.query(User, Site_info).filter(User.user_id == Site_info.user_id).all():
    print ("ID: {} Name: {} Invoice No: {} Amount: {}".format(c.user_id, c.full_name, i.address, i.site_name))
        



Method 2..

2022-04-01 05:34:24,588 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-04-01 05:34:24,589 INFO sqlalchemy.engine.base.Engine SELECT "User".user_id AS "User_user_id", "User".full_name AS "User_full_name", "User".contact_number AS "User_contact_number", "User".email_address AS "User_email_address", "User".username AS "User_username", "User".password AS "User_password", "Site_info".site_info_id AS "Site_info_site_info_id", "Site_info".site_name AS "Site_info_site_name", "Site_info".description AS "Site_info_description", "Site_info".contact_info AS "Site_info_contact_info", "Site_info".address AS "Site_info_address", "Site_info".last_update AS "Site_info_last_update", "Site_info".user_id AS "Site_info_user_id" 
FROM "User", "Site_info" 
WHERE "User".user_id = "Site_info".user_id
2022-04-01 05:34:24,591 INFO sqlalchemy.engine.base.Engine ()


# 4. Update records using SQLAlchemy ORM object
###  committ and rollback

### Updating customer's address by requesting new address from the customer

In [11]:
#adding customer_id from Customer table into a list(customer_ids) for exception handling
customer_ids=df['customer_id'].tolist()

while True:
    try:
        # Note: Python 2.x users should use raw_input, the equivalent of 3.x's input
        custom = int(input("Please enter your customer id: "))
    except ValueError:
        print("Sorry, Invalid customer id. Please check again!")
        #better try again... Return to the start of the loop
        continue
    else:
        #Customer_id was successfully parsed!
        #we're ready to exit the loop.
        break
#Using the list of customer_ids which we created to check for validity
if custom in customer_ids: 
    print("Customer id is valid.Please proceed to the next Step!")
else:
    print("Invalid customer id. Please sign up with our app if not registered")
    
# customer id(1-30) is valid as these are registered in our database.
#Try other values to check the error message given to customers


Please enter your customer id: 1
Customer id is valid.Please proceed to the next Step!


In [12]:


# The get(x) method of the Query object returns an object which holds the record associated with the primary key (x)
#custom has the customer_id which was entered by customer on the above code block
#Note :
record = objSession.query(Customer).get(custom)
# Displaying type of object returned
print(type(record))

record.customer_address = input('Enter your new address:')
# Display content of the object
print ("After update but before rollback...")
print ("ID:", record.customer_id, "Name:", record.customer_first_name, "Your new address:", record.customer_address, "Email:", record.customer_email)

# Note that we are not using committ but rollback. 
#We'll use rollback only if customer wants to undo the changes
#Using if statements to give user the option to rollback

#Requesting input to confirm if a rollback is needed for the Address change
test=(input('Do you want to UNDO the Address Change\n {y=Yes, n=no, q=quit} :'))

#While loop to get user to enter right options(y for yes, n for no and q for quit)
while test != 'y' and test != 'n' and test != 'q':
    test=(input('Please enter valid options provided to UNDO the Address Change and try again {y=Yes, n=no, q=quit} :'))

if  test == 'y' :
    objSession.rollback()
    print ("After rollback...")
    print ("ID:", record.customer_id, "Name:", record.customer_last_name, "Username:", record.customer_username, "Email:", record.customer_email)
    print('Rollback done successfully')
elif test == 'n':
    print('No changes done. Thank you!')
else :
    print('ADDRESS ROLLBACK SEQUENCE ENDED')
        
    


<class '__main__.Customer'>
Enter your new address:1
After update but before rollback...
ID: 1 Name: Adria  Your new address: 1 Email: vulputate.ullamcorper@aol.org
Do you want to UNDO the Address Change
 {y=Yes, n=no, q=quit} :y
2022-04-01 05:34:40,836 INFO sqlalchemy.engine.base.Engine ROLLBACK
After rollback...
2022-04-01 05:34:40,839 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-04-01 05:34:40,841 INFO sqlalchemy.engine.base.Engine SELECT "Customer".customer_id AS "Customer_customer_id", "Customer".customer_first_name AS "Customer_customer_first_name", "Customer".customer_last_name AS "Customer_customer_last_name", "Customer".customer_middle_name AS "Customer_customer_middle_name", "Customer".customer_email AS "Customer_customer_email", "Customer".customer_phone_number AS "Customer_customer_phone_number", "Customer".customer_landline AS "Customer_customer_landline", "Customer".profile_image AS "Customer_profile_image", "Customer".customer_username AS "Customer_customer_u

In [13]:
# using update() method of the Query object to make changes to several records (all records in our case)
# synchronize_session attribute mentions the strategy to update attributes in the session
objSession.query(Customer).update({Customer.customer_first_name:"Mr."+Customer.customer_first_name}, synchronize_session = False)
objSession.commit()

2022-04-01 05:34:45,736 INFO sqlalchemy.engine.base.Engine UPDATE "Customer" SET customer_first_name=(? || "Customer".customer_first_name)
2022-04-01 05:34:45,737 INFO sqlalchemy.engine.base.Engine ('Mr.',)
2022-04-01 05:34:45,740 INFO sqlalchemy.engine.base.Engine COMMIT


# Deleting Objects

In [15]:
from sqlalchemy.orm import sessionmaker

# Session class is defined using sessionmaker(), which is bound to the engine object created earlier
Session2 = sessionmaker(bind = engine)

# session object (objSession) is set-up using the default constructor
sessionObj2 = Session2()


# To delete record from a single table, you have to delete an object of the mapped class from a session and commit the action.
recordSet = sessionObj2.query(Customer).get(2)
print (recordSet.customer_id, recordSet.customer_first_name)

# Deleteing one record using .delete() method (pass object with primary key value = 2)
sessionObj2.delete(recordSet)

# Confirm to see if deleted (count will be zero)
sessionObj2.query(Customer).filter_by(customer_id = 2).count()

# However, related records present in Rating
sessionObj2.query(Rating).filter(Rating.remarks.in_([1,5])).count()

2022-04-01 06:25:16,985 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-04-01 06:25:16,987 INFO sqlalchemy.engine.base.Engine SELECT "Customer".customer_id AS "Customer_customer_id", "Customer".customer_first_name AS "Customer_customer_first_name", "Customer".customer_last_name AS "Customer_customer_last_name", "Customer".customer_middle_name AS "Customer_customer_middle_name", "Customer".customer_email AS "Customer_customer_email", "Customer".customer_phone_number AS "Customer_customer_phone_number", "Customer".customer_landline AS "Customer_customer_landline", "Customer".profile_image AS "Customer_profile_image", "Customer".customer_username AS "Customer_customer_username", "Customer".customer_password AS "Customer_customer_password", "Customer".account_status AS "Customer_account_status" 
FROM "Customer" 
WHERE "Customer".customer_id = ?
2022-04-01 06:25:16,988 INFO sqlalchemy.engine.base.Engine (2,)
2 Mr.Dorothy 
2022-04-01 06:25:16,995 INFO sqlalchemy.engine.base.Engine S

E:\Anaconda\lib\site-packages\sqlalchemy\sql\sqltypes.py:668: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


0

In [16]:
# DELETE RELATED RECORD FROM CHILD TABLE (CASCADE DELETE IN SQLite)
# To delete record from a single table, you have to delete an object of the mapped class from a session and commit the action.
recordSet = objSession.query(Customer).get(3)

# Deleteing one record using .delete() method (pass object with primary key value = 2)
objSession.delete(recordSet)

objSession.commit()

# Confirm to see if deleted (count will be zero)
objSession.query(Customer).filter_by(id = 2).count()

# However, related records present in Invoices
#objSession.query(InvoiceX).filter(InvoiceX.invno.in_([3,4])).count()

2022-04-01 06:25:30,958 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-04-01 06:25:30,959 INFO sqlalchemy.engine.base.Engine SELECT "Customer".customer_id AS "Customer_customer_id", "Customer".customer_first_name AS "Customer_customer_first_name", "Customer".customer_last_name AS "Customer_customer_last_name", "Customer".customer_middle_name AS "Customer_customer_middle_name", "Customer".customer_email AS "Customer_customer_email", "Customer".customer_phone_number AS "Customer_customer_phone_number", "Customer".customer_landline AS "Customer_customer_landline", "Customer".profile_image AS "Customer_profile_image", "Customer".customer_username AS "Customer_customer_username", "Customer".customer_password AS "Customer_customer_password", "Customer".account_status AS "Customer_account_status" 
FROM "Customer" 
WHERE "Customer".customer_id = ?
2022-04-01 06:25:30,960 INFO sqlalchemy.engine.base.Engine (3,)
2022-04-01 06:25:30,963 INFO sqlalchemy.engine.base.Engine SELECT "Rating"

OperationalError: (sqlite3.OperationalError) database is locked
[SQL: DELETE FROM "Customer" WHERE "Customer".customer_id = ?]
[parameters: (3,)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

# Dropping  All the tables

In [17]:
# All tables can be deleted using the drop_all() method. 
# This method does the exact opposite of create_all()
# When drop_all() command is issued, the presence of each table is checked first, and tables are dropped in reverse order of dependency.

Base.metadata.drop_all(engine)

2022-04-01 06:25:41,574 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Customer")
2022-04-01 06:25:41,575 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 06:25:41,577 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("User")
2022-04-01 06:25:41,578 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 06:25:41,579 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Site_info")
2022-04-01 06:25:41,580 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 06:25:41,582 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("link")
2022-04-01 06:25:41,582 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 06:25:41,585 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Order")
2022-04-01 06:25:41,585 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 06:25:41,587 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Payment_info")
2022-04-01 06:25:41,588 INFO sqlalchemy.engine.base.Engine ()
2022-04-01 06:25:41,589 INFO sqlalchemy.engine.base.Engine PRAGMA main

OperationalError: (sqlite3.OperationalError) database is locked
[SQL: 
DROP TABLE "Rating"]
(Background on this error at: http://sqlalche.me/e/13/e3q8)